# Analyse large "covid" shock events
* plot searching patients within one shock
* save in table format

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
### define shock size and physicians
shock = 15
doclist = list(['AM','IM','KI','PSY','ORTR','RAD','DER','URO','HNO','CH','NEU','AU','GGH'])

### define for renaming specialist in english
doc_list = {'DE':['AM', 'AU', 'CH', 'DER', 'GGH', 'HNO', 'IM', 'KI', 'NEU', 'ORTR','PSY', 'RAD', 'URO'],
            'EN':['GP','OPH','SRG','DER','OBGYN','ENT','IM','PED','NEU','ORTH','PSY','RAD','URO']}

In [ ]:
all_dta = pd.DataFrame()

### import sim results
for doc in doclist:    
    dta = pd.read_csv('results/searching_pats_{}_iter10_shocksize{}.csv'.format(doc,shock),header=None)
    dta = dta.astype(int)

    ### import general info on patient numbers
    pats = pd.read_excel('results/states_doc_info_total_hour-based_quarterly.xlsx')

    pats = pd.DataFrame(pats.sum(axis=0)).T

    ### switch to long format for seaborn plotting
    dta2 = pd.DataFrame(columns=['shock','searching','at_doc','at_doc_rel','speciality','speciality_en'])
    for i in range(len(dta)):
        dta_ad = pd.DataFrame()
        dta_ad['searching'] = dta.loc[:,i].copy()
        dta_ad['at_doc'] = pats.loc[0,'{}_total'.format(doc)] - dta_ad['searching']
        dta_ad['at_doc_rel'] = dta_ad['at_doc'] / pats.loc[0,'{}_total'.format(doc)] * 100
        dta_ad['shock'] = i
        dta_ad['speciality'] = doc
        dta_ad['speciality_en'] = doc_list['EN'][doc_list['DE'].index(doc)]
        dta2 = pd.concat([dta2,dta_ad])

    ### add pre-shock points
    dta2 = dta2.append({'shock':-1,'searching':0,'at_doc':pats.loc[0,'{}_total'.format(doc)],
                 'at_doc_rel':100,'speciality':doc,'speciality_en':doc_list['EN'][doc_list['DE'].index(doc)]},ignore_index=True)
    dta2 = dta2.append({'shock':-2,'searching':0,'at_doc':pats.loc[0,'{}_total'.format(doc)],
                 'at_doc_rel':100,'speciality':doc,'speciality_en':doc_list['EN'][doc_list['DE'].index(doc)]},ignore_index=True)
    dta2 = dta2.append({'shock':-3,'searching':0,'at_doc':pats.loc[0,'{}_total'.format(doc)],
                 'at_doc_rel':100,'speciality':doc,'speciality_en':doc_list['EN'][doc_list['DE'].index(doc)]},ignore_index=True)
    
    all_dta = pd.concat([all_dta,dta2])

In [ ]:
### plot all specialists in single graphs with gray lines of all other docs for comparison

figs, axs = plt.subplots(5,3,figsize=(11,14))
r = 0
c = 0
### import sim results
for doc in doclist:
    if (c==3):
        r+=1; c=0    
    doc_en = doc_list['EN'][doc_list['DE'].index(doc)]

    ### highlight only one specialist
    colors = ['lightgray' for x in range(len(doclist))]
    colors[-1] = 'blue'
    
    ### get docs in correct order
    a = list(all_dta.speciality.unique())
    a.remove(doc)
    a.append(doc)

    sns.lineplot(x='shock',y='at_doc_rel',data = all_dta,hue = 'speciality',hue_order=a,
                 palette = colors,linewidth=.8,ax = axs[r,c])
    if c==0:
        axs[r,c].set_ylabel('At physician [%]',fontsize=16)
    else:
        axs[r,c].set_ylabel('')
        
    axs[r,c].set_xlabel('Displacement steps',fontsize=16)
    if (r==4)&((c==1)|(c==2)):
        axs[r,c].set_xlabel('')
    
    axs[r,c].tick_params(axis='y', labelsize=12)
    axs[r,c].set_xticks(np.arange(-3,10,1))
    axs[r,c].set_xticklabels(['','pre-shock','',1,2,3,4,5,6,7,8,9,10],fontsize=12)
    axs[r,c].axvline(-1,color='r',alpha=0.8)
    axs[r,c].axhline(100,color='k',alpha=0.8,linestyle='--')
    min_y = 88
    axs[r,c].text(x=-.9,y=min_y+0.5,s='shock',fontsize=16,color='r')
    axs[r,c].set_ylim([min_y,100.7])
    axs[r,c].set_title('{}'.format(doc_en),fontsize=16)
    axs[r,c].get_legend().remove()
    
    c+=1
    
axs[-1, -1].axis('off')
axs[-1, -2].axis('off')

plt.suptitle('Resilience curves - shock size = {}%'.format(shock),fontsize=18,y=0.99)
plt.tight_layout()
plt.savefig('figures/covid_shock_grays_size{}.png'.format(shock),dpi=250,format='png')
plt.savefig('figures/covid_shock_grays_size{}.svg'.format(shock),format='svg')
plt.show()

## Create table with averaged values
* mean and std over 10 iterations for each specialty

In [ ]:
all_dta['rel_searching'] = 100-all_dta['at_doc_rel']

lst_all = []
for doc in all_dta.speciality_en.unique():
    fin_tab_ad = pd.DataFrame(columns=['specialty','start_mean','start_std','end_mean','end_std'])
    start_mean = all_dta.loc[(all_dta.speciality_en==doc)&(all_dta.shock==0),'rel_searching'].mean()
    start_std = all_dta.loc[(all_dta.speciality_en==doc)&(all_dta.shock==0),'rel_searching'].std()
    
    end_mean = all_dta.loc[(all_dta.speciality_en==doc)&(all_dta.shock==9),'rel_searching'].mean()
    end_std = all_dta.loc[(all_dta.speciality_en==doc)&(all_dta.shock==9),'rel_searching'].std()
    
    lst = list([doc,start_mean,start_std,end_mean,end_std])
    lst_all.append(lst)

lst_all = pd.DataFrame(lst_all,columns=['specialty','step0_mean_perc','step0_std_perc','step9_mean_perc','step9_std_perc'])
lst_all.to_csv('results/shock_table_size{}.csv'.format(shock),index=False)

In [ ]:
lst_all